# Коллаборативная фильтрация

Полезная ссылка на рекомендательную систему на Kaggle:

https://www.kaggle.com/sjj118/movie-visualization-recommendation-prediction

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

# Подготовка данных

In [2]:
path = 'D:\\myIT\\myProjects\\sberbank\\recomendation_system_181120\\the_movies_dataset\\'

ratings = pd.read_csv(path + 'ratings_small.csv', usecols=['userId', 'movieId', 'rating'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


У тебя появился доступ к идентификатору пользователя (userId), идентификатору фильма (movieId) и к рейтинговой оценке (rating), которую пользователь поставил конкретному фильму.

Чтобы увидеть названия фильмов, которые посмотрел и оценил пользователь, возьмем данные нашего основного файла «movies_metadata_fixed.csv». Загрузи названия кино (title) и объедини два датасета методом **merge** по столбцам **movieId** слева и **id** справа, а затем удали дубликат столбца **id**.

In [3]:
#Загрузи из movies_metadata_fixed.csv столбцы ['id', 'title']
movies = pd.read_csv(path + 'movies_metadata_fixed.csv', usecols=['id', 'title'])

#Объедини таблицы при помощи метода merge
info = pd.merge(ratings, movies, left_on='movieId', right_on='id').sort_values('userId')
#Удали столбец-дубликат
info = info.drop('id', axis=1)
info.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
182,1,2294,2.0,Jay and Silent Bob Strike Back
235,1,2455,2.5,Confidentially Yours
47,1,1405,1.0,Greed
140,1,2193,2.0,My Tutor


Чтобы дальше не путаться в идентификационных номерах, немного изменим их. Фильмов в основном наборе данных по количеству больше, чем в сокращенном файле «ratings_small.csv». Это значит, что в колонке **movieId** будут пропуски в порядке номеров. Например, изначально у нас есть 100 фильмов, но мы случайным образом в датасет отобрали 5 фильмов с идентификаторами: 1, 3, 12, 55, 79. Будет гораздо удобнее, если в объединенном датасете id этих фильмов будут: 1, 2, 3, 4, 5. Поэтому сейчас мы заново присвоим id фильмам в объединенном датасете, чтобы нумерация совпадала с числом фильмов.

In [4]:
movie_ids = info['movieId'].unique()

def scale_movie_id(movie_id):
    scaled = np.where(movie_ids == movie_id)[0][0] + 1
    return scaled

info['movieId'] = info['movieId'].apply(scale_movie_id)

user_ids = info['userId'].unique()

def scale_user_id(user_id):
    scaled = np.where(user_ids == user_id)[0][0] + 1
    return scaled

info['userId'] = info['userId'].apply(scale_user_id)

n_movies = len(info['movieId'].unique())
n_users = len(info['userId'].unique())

info.head()

,userId,movieId,rating,title
0,1,1,2.5,Rocky III
182,1,2,2.0,Jay and Silent Bob Strike Back
235,1,3,2.5,Confidentially Yours
47,1,4,1.0,Greed
140,1,5,2.0,My Tutor


In [5]:
n_users

671

In [6]:
n_movies

2830

In [7]:
info.shape

(44994, 4)

Теперь ты точно не пропустишь фильм и не запутаешься в идентификаторах при работе с кодом твоей программы.

# Предсказание рейтинговых оценок

Теперь тебе нужно разделить получившийся набор данных на две части: обучающую (train) и тестовую (test) выборку. Когда мы делаем предсказание, наша задача — получить честный прогноз. Как мы узнаем, что предсказания оценок близки к действительным оценкам? Мы можем обучить модель на одной части нашего набора данных и проверить на второй, как хорошо модель справляется с угадыванием оценок. Поэтому первую часть выборки ты используешь для обучения, а на второй части измеришь качество предсказанных оценок. 

Чтобы разделить датасет на две части случайным образом, используй фукцию **train_test_split** из библиотеки **scikit-learn**:

In [8]:
#Загрузи библиотеку scikit-learn и разбей выборку на две: train и test
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(info, test_size=0.2)

print('Train shape: ', train_data.shape)
print('Test shape: ', test_data.shape)

Train shape:  (35995, 4)
Test shape:  (8999, 4)


Определять качество предсказанных оценок будем с помощью показателя ошибки модели — RMSE.

Давай напишем функцию, чтобы проводить оценку:

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    # Оставь оценки, предсказанные алгоритмом, только для нужного набора данных
    prediction = np.nan_to_num(prediction)[ground_truth.nonzero()].flatten()
    # Оставь действительные оценки пользователей только для соотвествующего набора данных
    ground_truth = np.nan_to_num(ground_truth)[ground_truth.nonzero()].flatten()
    
    mse = mean_squared_error(prediction, ground_truth)
    return sqrt(mse)

А теперь нужно подготовить и сформировать матрицы
- для тренировочной выборки с действительными оценками пользователей 
- для обучающей выборки с предсказанными оценками

Обе матрицы должны быть **размера (n_users, n_movies)**, чтобы элемент в ячейке **[i,j]** отражал оценку **i-го** пользователя **j-му** фильму:

In [10]:
train_data.head(3)

,userId,movieId,rating,title
38981,624,1170,3.0,Very Annie Mary
12370,303,90,3.5,Fools Rush In
4692,497,44,5.0,Once Were Warriors


In [11]:
# строка датафрейма берется как кортеж, 
# в ячейку userId-movieId (индексы [line[1] - 1, line[2] - 1]) записываем оценку rating (line[3])
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

In [12]:
train_data_matrix.shape

(671, 2830)

Напомним, что часть оценок попала в обучающую выборку **train_data_matrix**, а другая часть в тестовую — **test_data_matrix**, чтобы можно было оценить качество предсказаний.

Дальше, нужно понять, как именно предсказывать оценки. В системах коллаборативной фильтрации для этого используют два подхода:
- user-based или «рекомендации, основанные на пользователях», когда мы ищем похожих пользователей
- item-based или «рекомендации, основанные на объектах», когда мы ищем похожие объекты, то есть похожие фильмы в задании

Осталось понять, как реализовать поиск похожих пользователей и фильмов, как мы это будем определять? Есть разные подходы, один из них — использовать **косинусное расстояние** между векторами, в которых хранится описание пользователей и фильмов. Косинусное растояние считается по формуле: 

$$similarity = cos(\theta) = \dfrac{A \cdot B}{||A|| \cdot ||B||} = \dfrac{\sum (A_i \cdot B_i)}{\sqrt{\sum (A_i)^2} \cdot \sqrt{\sum (B_i)^2}} $$


- $A$ - первый вектор с оценками зрителя № 1 за все фильмы
- $B$ - второй вектор с оценками зрителя № 2 за все фильмы
- $||A||$ - длина вектора A, это сумма квадратов элементов под корнем.
- $||B||$ - длина вектора B, это сумма квадратов элементов под корнем.
- $A \cdot B$ - сумма поэлементного произведения векторов (скалярное произведение векторов).
Например, вектор оценок A = {1, 2, 5}, вектор оценок B = {3, 4, 2}
Тогда скалярное произведение мы вычисляем так: $A \cdot B = 1 \cdot 3 + 2 \cdot 4 + 5 \cdot 2 = 21$

Хорошие новости — в библиотеке **Scikit-learn** уже есть готовая функция **pairwise_distances**:

In [13]:
from  sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов 
# (по строкам и по колонкам соотвественно).
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

В коде:
- user_similarity[i][j] — косинусное расстояние между i-ой строкой и j-ой строкой;
- item_similarity[i][j] — косинусное расстояние между i-ой и j-ой колонками.

Будем считать, что косинусное расстояние обозначает степень похожести: **чем пользователи или фильмы более похожи друг на друга — тем меньше будет косинусное расстояние**. Посмотри, как это работает на небольшом примере. Представим, у нас есть оценки твоих знакомых для нескольких фильмов, соберем их в один датафрейм:

In [14]:
#создаем датафрейм со случайными оценками
d = {"Хоббит":         pd.Series([0, 4, 0, 4], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Мстители":       pd.Series([5, 5, 0, 5], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Человек-паук":   pd.Series([1, 0, 1, 2], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Матрица":        pd.Series([5, 4, 2, 3], index=['Маша', 'Миша', 'Ваня', 'Настя']),
     "Звездные войны": pd.Series([0, 3, 1, 4], index=['Маша', 'Миша', 'Ваня', 'Настя'])}
df1 = pd.DataFrame(d)

display(df1)

,Хоббит,Мстители,Человек-паук,Матрица,Звездные войны
Маша,0,5,1,5,0
Миша,4,5,0,4,3
Ваня,0,0,1,2,1
Настя,4,5,2,3,4


Из таблицы становится ясно, что Маша ещё не смотрела "Хоббита" и "Звездные войны". Теперь создаем датафрейм, где сохраним косинусное расстояние между ребятами. 

In [15]:
dist = pairwise_distances(df1, metric='cosine')
dist = np.round(dist, 3)
dist_df = pd.DataFrame(data=dist, index = ['Маша', 'Миша', 'Ваня', 'Настя'], columns=['Маша', 'Миша', 'Ваня', 'Настя'])
display(dist_df)

,Маша,Миша,Ваня,Настя
Маша,0.000,0.224,0.371,0.297
Миша,0.224,0.000,0.447,0.044
Ваня,0.371,0.447,0.000,0.414
Настя,0.297,0.044,0.414,0.000


Результат смотрим по строке и узнаем, что два самых похожих на Машу пользователя (по косиносному расстоянию) — это Миша (0.224) и Настя (0.297). 

Дальше ты попробуешь использовать оба подхода для написания программы:
- User-based коллаборативная фильтрация (предскажем оценки фильмам на основе "похожести" пользователей)
- Item-Based коллаборативная фильтрация (предскажем оценки фильмам на основе "похожести" фильмов). 

Подходы отличаются только матрицами оценок, алгоритм остается одинаковым. Если в User-based подходе фильмы располагаются по столбцам, а пользователи по строкам, то в Item-Based подходе фильмы располагаются в строках, а пользователи — в колонках. Мы с тобой реализуем три варианта систем коллаборативной фильтрации и в каждой попробуем использовать оба подхода, но в разных форматах.

### Первый вариант модели

Начнём с самого простого алгоритма коллаборативной фильтрации. Оценка, которую мы хотим предсказать — это среднее арифмитическое оценок этого фильма от зрителей, похожих на нашего пользователя.

Предсказанная оценка(**u**) определяется так:  оценка **u** по фильму **i** равна средней оценке фильма **i** от **N** пользователей, которые больше всего похожи на пользователя **u**. А похожих пользователей мы отобрали выше, вычислив косинусное расстояние между ними.

Для вычисления оценки мы используем следующую формулу:

$$  R = \cfrac{\sum r_k}{N} $$

- $U$ - пользователь, для которого предсказыааем оценку
- $i$ - фильм, для которого предсказываем оценку пользователя U
- $R$ - предсказанная оценка пользователя U за фильм i 
- $r_k$ - оценка пользователя k, который похож на пользователя U, за фильм i
- $\sum r_k $ - сумма оценок всех похожих на пользователя U зрителей за фильм i
- $N$ - количество зрителей, похожих на пользователя U


Помнишь пример выше, где мы рассчитали косинусное расстояние между зрителями? Маша не смотрела "Хоббита" и "Звездные войны", вот как бы мы предсказали её оценки вручную. Напомним, самые похожие зрители с Машей это Миша и Настя, поэтому мы берем их оценки для расчёта:

**Предсказанная оценка Маши для фильма "Хоббит"**

- оценка Мишы за фильм "Хоббит" = 4
- оценка Насти за фильм "Хоббит" = 4

$  R_{Маша, Хоббит} = \cfrac{4 + 4}{2} = 4$

**Предсказанная оценка Маши для фильма "Звездные войны"**

- оценка Мишы за фильм "Звездные войны" = 3
- оценка Насти за фильм "Звездные войны" = 4

$  R_{Маша, ЗВ} = \cfrac{3 + 4}{2} = 3.5$

Теперь проделаем такой же расчёт для твоего основного датасета в задании: 

In [16]:
user_similarity[0].argsort()[1:7 + 1]

array([ 34, 324, 633, 476,  40, 345,   6], dtype=int64)

In [18]:
# User-based коллаборативная фильтрация
def naive_predict(top):
    # Структура хранения оценки фильмов от самых похожих пользователей для каждого зрителя
    # (количество похожих пользователей хранится в переменной top):
    # top_similar_ratings[0][1] - оценки всех фильмов одного из самых похожих пользователей на зрителя с id 0.
    # Здесь 1 - это не id пользователя, а просто порядковый номер.
    top_similar_ratings = np.zeros((n_users, top, n_movies))

    for i in range(n_users):
        # Для каждого зрителя необходимо найти наиболее похожих пользователей:
        # нулевой элемент не учитываем, так как на этом месте хранится похожесть пользователя самого на себя
        top_sim_users = user_similarity[i].argsort()[1:top + 1] #здесь в top_sim_users мы записываем id пользователей, у которых косинусные расстояния самые маленькие до текущего пользователя i
        
        # берём только оценки из "обучающей" выборки 
        top_similar_ratings[i] = train_data_matrix[top_sim_users] #создаём таблицу для каждого пользователя, в которой хранятся оценки за все фильмы от похожих пользователей

    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top #применяем формулу, описанную выше. 
        #top_similar_ratings[i].sum(axis=0) - это сумма оценок "похожих" пользователей, top - количество "похожих" пользователей
     
    return pred


def naive_predict_item(top):
    top_similar_ratings = np.zeros((n_movies, top, n_users))

    for i in range(n_movies):
        top_sim_movies = item_similarity[i].argsort()[1:top + 1] #находим наиболее близкие фильмы по оценкам

        top_similar_ratings[i] = train_data_matrix.T[top_sim_movies] #создаём таблицу для каждого фильма, в которой хранятся оценки всех пользователей за похожие фильмы 
        
    pred = np.zeros((n_movies, n_users))
    for i in range(n_movies):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top #применяем формулу, описанную выше, только для фильмов, а не пользователей 
    
    return pred.T

naive_pred = naive_predict(7) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(naive_pred, test_data_matrix))

naive_pred_item = naive_predict_item(7) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(naive_pred_item, test_data_matrix))

User-based CF RMSE:  2.7986936341957986
Item-based CF RMSE:  2.911539468595995


### Второй вариант модели

Попробуем посторить другой вариант алгоритма, он будет опираться на две матрицы: "похожести пользователей" и оценок "похожих" пользователей. В этом варианте мы ожидаем, что похожие зрители будут ставить примерно похожие оценки фильмам, и так алгоритм сможет предсказать оценки. Для этого мы используем следующую формулу:

$$R = \cfrac{\sum (Sim_k \cdot r_k)}{\sum Sim}$$

- $R$ - предсказанная оценка пользователя U за фильм i 
- $Sim_k$ - "похожесть" пользователя k на пользователя U, в нашем случае, это косинусное расстояние
- $r_k$ - оценка пользователя k за фильм i
- $\sum Sim $ - сумма косинусных расстояний всех похожих пользователей

Помнишь из примера выше, что Маша ещё не смотрела фильм "Хоббит" и "Звездные войны"? Стоит ли посоветовать ей это кино? Мы должны предсказать её оценку, чтобы дать удачную рекомендацию. Из нашей матрицы "похожести пользователей" мы вновь берем зрителей - Мишу и Настю, они похожи на Машу больше других по косинусному расстоянию. В этой модели, чем больше похожи пользователи, тем сильнее оценка такого зрителя влияет на предсказанную оценку.


**Для фильма "Хоббит"**

- оценка Мишы за фильм = 4
- расстояние до Миши = 0.224


- оценка Насти за фильм  = 4
- расстояние до Насти = 0.297

**Предсказание оценки Маши для фильма "Хоббит"**

$  R_{Маша, Хоббит} = \cfrac{0.224 \cdot 4 + 0.297 \cdot 4}{0.224 + 0.297} = \cfrac{2.084}{0.521} = 4$


**Для фильма "Звездные войны"**

- оценка Мишы за фильм = 3
- расстояние до Миши = 0.224


- оценка Насти за фильм  = 4
- расстояние до Насти = 0.297

**Предсказание оценки Маши для фильма "Звездные войны"**

$  R_{Маша, ЗВ} = \cfrac{0.224 \cdot 3 + 0.297 \cdot 4}{0.224 + 0.297} = \cfrac{1.86}{0.521} = 3.57$

 Посмотрим, как реализовать вторую модель для твоего основного набора данных:

In [20]:
def k_fract_predict(top):
    top_similar = np.zeros((n_users, top))
    
    for i in range(n_users): #создаём массив, в котором будем хранить для каждого пользователя id "похожих" на него пользователей
        user_sim = user_similarity[i]
        top_sim_users = user_sim.argsort()[1:top + 1]#[-top:]

        top_similar[i] = top_sim_users
            
    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int) #записываем id людей, "похожих" на пользователя i в отдельную переменную
        numerator = user_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" людей для пользователя i
        
        product = np.dot(numerator, train_data_matrix[indexes]) #Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" пользователей
        #вторая матрица содержит оценки этих пользователей за все фильмы
        
        denominator = numerator.sum() #сумма расстояний до "похожих" пользователей, наш знаменатель
        
        pred[i] = product / denominator #вычисляем значения для нашего вектора с предсказанными оценками пользователя i для всех фильмов
    
    return pred


def k_fract_predict_item(top):
    top_similar = np.zeros((n_movies, top))
    
    for i in range(n_movies): #создаём массив, в котором будем хранить для каждого фильма id "похожих" на него фильмов
        movies_sim = item_similarity[i]
        top_sim_movies = movies_sim.argsort()[1:top + 1]

        top_similar[i] = top_sim_movies.T
            
    pred = np.zeros((n_movies, n_users))
    
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int) #записываем id фильмов, "похожих" на фильм i в отдельную переменную
        numerator = item_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" фильмов для фильма i 
        
        product = np.dot(numerator, train_data_matrix.T[indexes])#Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" фильмов
        #вторая матрица содержит оценки всех пользователей за "похожие" фильмы
        
        denominator = numerator.sum() #сумма расстояний до "похожих" фильмов, наш знаменатель
        
        pred[i] = product / denominator #вычисляем значения для нашего вектора с предсказанными оценками за фильм i для всех пользователей
        
    return pred.T


k_predict = k_fract_predict(7) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(k_predict, test_data_matrix))

k_predict_item = k_fract_predict_item(7) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(k_predict_item, test_data_matrix))

User-based CF RMSE:  2.799920346052418
Item-based CF RMSE:  3.0025760769134684


In [21]:
k_predict

array([[3.13938332, 0.        , 0.15415952, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.43689037, 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Третий вариант модели

Третий вариант, который ты дальше попробуешь запрограммировать, зависит от 1) среднего значения всех оценок, которые зритель до этого оставил фильмам, 2) от средних оценок "похожих" пользователей" и 3) от коэффициентов “похожести” зрителей. 

Для этого мы используем следующую формулу:

$$  R = \overline{R} + \cfrac{\sum \left(Sim_k \cdot (r_k - \overline{r_k})\right)}{\sum Sim} $$

- $R$ - предсказанная оценка пользователя U за фильм i 
- $\overline{R}$ - средняя оценка пользователя U по всем фильмам, которые он смотрел
- $Sim_k$ - "похожесть" пользователя k на пользователя U, в нашем случае, это косинусное расстояние
- $r_k$ - оценка пользователя k за фильм i
- $\overline{r_k}$ - средняя оценка пользователя k по всем фильмам, которые он смотрел
- $\sum Sim $ - сумма косинусных расстояний всех похожих пользователей

Давай вернемся к примеру. Мы все ещё хотим узнать, как бы Маша оценила фильмы "Хоббит" и "Звездные войны". Из истории просмотров мы знаем, что Миша и Настя самые похожие на Машу зрители.

Предскажем оценку Маши для двух фильмов:

**Для начала посчитаем средние оценки по всем фильмам для каждого пользователя**

- средняя оценка Маши за все фильмы = $\cfrac{5 + 1 + 5}{3} = 3.67$
- средняя оценка Миши за все фильмы = $\cfrac{4 + 5 + 4 + 3}{4} = 4$
- средняя оценка Насти за все фильмы = $\cfrac{4 + 5 + 2 + 3 + 4}{5} = 3.6$

**Для фильма "Хоббит"**

- оценка Мишы за фильм = 4
- расстояние до Миши = 0.224

- оценка Насти за фильм  = 4 
- расстояние до Насти = 0.297

**Предсказание оценки Маши для фильма "Хоббит"**

$  R_{Маша, Хоббит} = 3.67 + \cfrac{0.224 \cdot (4-4) + 0.297 \cdot (4-3.6)}{0.224 + 0.297} = 3.67 + \cfrac{0.1188}{0.521} = 3.898$

**Для фильма "Звездные войны"**

- оценка Мишы за фильм = 3
- расстояние до Миши = 0.224

- оценка Насти за фильм  = 4
- расстояние до Насти = 0.297

**Предсказание оценки Маши для фильма "Звездные войны"**

$  R_{Маша, ЗВ} = 3.67 + \cfrac{0.224 \cdot (3-4) + 0.297 \cdot (4-3.6)}{0.224 + 0.297} = 3.67 + \cfrac{-0.1052}{0.521} = 3.47$

Мы получили предполагаемые оценки Маши за два фильма. Вернись к заданию и попробуй использовать идею расчётов для третьей модели на основном наборе данных:

In [24]:
def k_fract_mean_predict(top):
    top_similar = np.zeros((n_users, top))
    
    for i in range(n_users): #создаём массив, в котором будем хранить для каждого пользователя id "похожих" на него пользователей
        user_sim = user_similarity[i]
        top_sim_users = user_sim.argsort()[1:top + 1]

        top_similar[i] = top_sim_users

    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int) #записываем id людей, "похожих" на пользователя i в отдельную переменную
        numerator = user_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" людей для пользователя i
        
        mean_rating = np.array([x for x in train_data_matrix[i] if x > 0]).mean() #вычисляем средний рейтинг пользователя i по всем фильмам, которые он посмотрел, если рейтинг равен 0, то не учитываем его
        
        diff_ratings = train_data_matrix[indexes] - train_data_matrix[indexes].mean() #находим разность между оценкой за конкретный фильм и средней оценкой по всем фильмам для всех "похожих" пользователей
        
        product = np.dot(numerator, diff_ratings) #Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех пользователей, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" пользователей
        #вторая матрица содержит результаты вычитания средней оценки из реальной оценки этих пользователей за все фильмы
        
        denominator = numerator.sum() #сумма расстояний до "похожих" пользователей, наш знаменатель
      
        pred[i] =  mean_rating + product / denominator #вычисляем значения для нашего вектора с предсказанными оценками пользователя i для всех фильмов
        
    return pred



def k_fract_mean_predict_item(top):
    top_similar = np.zeros((n_movies, top))
    
    for i in range(n_movies): #создаём массив, в котором будем хранить для каждого фильма id "похожих" на него фильмов
        movie_sim = item_similarity[i]
       
        top_sim_movies = movie_sim.argsort()[1:top + 1]
        
        top_similar[i] = top_sim_movies
           
    pred = np.zeros((n_movies, n_users))
    
    for i in range(n_movies):
        indexes = top_similar[i].astype(np.int) #записываем id фильмов, "похожих" на фильм i в отдельную переменную
        numerator = item_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" фильмов для фильма i 

        mean_rating = np.array([x for x in train_data_matrix.T[i] if x > 0]).mean() #вычисляем средний рейтинг фильма i по всем пользователям, которые его посмотрели
        
        diff_ratings = train_data_matrix.T[indexes] - train_data_matrix.T[indexes].mean() #находим разность между оценкой конкретного пользователя и средней оценкой по всем пользователям для всех "похожих" фильмов
        
        product = np.dot(numerator, diff_ratings) #Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех фильмов, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" фильмов
        #вторая матрица содержит результаты вычитания средней оценки из реальной оценки для этих фильмов от всех пользователей
        
        denominator = numerator.sum() #сумма расстояний до "похожих" фильмов, наш знаменатель
        
        pred[i] = mean_rating + product / denominator #вычисляем значения для нашего вектора с предсказанными оценками для фильма i по всем пользователям
        
    return pred.T

k_predict = k_fract_mean_predict(25) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(k_predict, test_data_matrix))

k_predict_item = k_fract_mean_predict_item(25) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(k_predict_item, test_data_matrix))

User-based CF RMSE:  1.3970299115503924
Item-based CF RMSE:  1.3545026504487854


Если захочется выгрузить с переименованием строк-столбцов как в kaggle соревновании, то делаем вот так:

In [28]:
# solution = pd.DataFrame(k_predict)
# # 671 пользовательских оценок (строки) по 2830 фильмам (столбцы)
# solution.head()

# # переименовываем столбцы и строки как требуется в форме для каггла
# col = []
# for  i in range(n_movies):
#     col.append('movie '+str(i))

# solution.index.names = ['id'] 
# solution.columns = col

# # выгружаем в csv файл
# solution.to_csv('collaboration_rec_system_191120_solution.csv')

# Вместо заключения


Ты дошёл до конца финального проекта по созданию рекомендательных систем. Поздравляем! Гордимся тобой, ведь у тебя получилось решишь реальную практическую задачу data science профессионала: ты разработал целых три типа систем рекомендаций. Простую неперсональную систему, контентную и систему коллаборативной фильтрации. Какую из них использовать? Это вопрос, на который правильный ответ знаешь только ты. Решение зависит от конкретных условий, ведь всё, что мы запрограммировали, точно используется сегодня в сфрере аналитики данных. 

**Теперь ты можешь доработать последнюю модель и посоревноваться с другими участниками в лучшем решении в конкурсе на Kaggle**:
***www.kaggle.com/c/ai-academy-movie-rating-competition***